In [55]:
#Initial imports 

import os 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly.express as px 
import hvplot.pandas 
from dotenv import load_dotenv 

%matplotlib inline

In [56]:
#Read the Mapbox API key 

load_dotenv()
mapbox_token = os.getenv("mapbox_api")
px.set_mapbox_access_token(mapbox_token)

In [57]:
#Read the census data into a Pandas DataFrame 

sfo_data = pd.read_csv("sfo_neighborhoods_census_data.csv", index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


Housing Units Per Year

In [58]:
#Calculate the mean number of housing units per year 

sfo_data_average = sfo_data[["housing_units"]].groupby([sfo_data.index]).mean()
sfo_data_average

,housing_units
year,
2010,372560
2011,374507
2012,376454
2013,378401
2014,380348
2015,382295
2016,384242


In [59]:
#Use the Pandas plot function to plot the average housing units per year 
#Note: You will need to manually adjust the y limit of the chart using the min and max values from above 


sfo_data_average.hvplot.bar(x="year",y="housing_units", xlim=(2010,2016), ylim=(350000,385000))

:Bars   [year]   (housing_units)

Average Prices per Square Foot 

In [60]:
#Calculate the average gross rent and average sale price per square foot 

sfo_data_average_2 = sfo_data[["sale_price_sqr_foot","gross_rent"]].groupby([sfo_data.index]).mean()
sfo_data_average_2

,sale_price_sqr_foot,gross_rent
year,,
2010,369.344353,1239
2011,341.903429,1530
2012,399.389968,2324
2013,483.600304,2971
2014,556.277273,3528
2015,632.540352,3739
2016,697.643709,4390


In [61]:
#Plot the Average Gross Rent per Year as a Line chart 

sfo_data_average_2.hvplot(x="year", y="gross_rent", title="Average Gross Rent in SF")


:Curve   [year]   (gross_rent)

In [62]:
#plot the Average Sales Price per Year as line chart 

sfo_data_average_2.hvplot(x="year", y="sale_price_sqr_foot", title="Average Sale Price per Square Foot in SF")

:Curve   [year]   (sale_price_sqr_foot)

Average Prices per Neighborhood 

In [63]:
#Group by year and neighborhood and then create a new dataframe of the mean 


sfo_data = sfo_data.reset_index()



In [96]:
df2 = sfo_data.groupby(["year","neighborhood"]).mean()
df2

sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                          
2010 Alamo Square                     291.182945         372560        1239
     Anza Vista                       267.932583         372560        1239
     Bayview                          170.098665         372560        1239
     Buena Vista Park                 347.394919         372560        1239
     Central Richmond                 319.027623         372560        1239
...                                          ...            ...         ...
2016 Telegraph Hill                   903.049771         384242        4390
     Twin Peaks                       970.085470         384242        4390
     Van Ness/ Civic Center           552.602567         384242        4390
     Visitacion Valley                328.319007         384242        4390
     Westwood Park                    631.195426         384242        4390

[397 rows x 3 columns]

In [97]:
#Use hvplot to create an interactive line chart of the average price per square foot 
#The plot should have a dropdown selector for the nieghborhood 

df2.hvplot.line(x='year', y='sale_price_sqr_foot', groupby="neighborhood", hover_cols='all')

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot,neighborhood,housing_units,gross_rent)

The Top 10 Most expensive neighborhoods 

In [145]:
#Getting the data for the top 10 most expensive neighborhoods 

df3 = sfo_data.groupby(["neighborhood"]).mean()
top_10_neighborhoods = df3.nlargest(10, "sale_price_sqr_foot")
top_10_neighborhoods

,sale_price_sqr_foot,housing_units,gross_rent
neighborhood,,,
Union Square District,903.993258,377427.50,2555.166667
Merced Heights,788.844818,380348.00,3414.000000
Miraloma Park,779.810842,375967.25,2155.250000
Pacific Heights,689.555817,378401.00,2817.285714
Westwood Park,687.087575,382295.00,3959.000000
Telegraph Hill,676.506578,378401.00,2817.285714
Presidio Heights,675.350212,378401.00,2817.285714
Cow Hollow,665.964042,378401.00,2817.285714
Potrero Hill,662.013613,378401.00,2817.285714


In [146]:
#Plotting the data from the top 10 expensive neighborhoods 

top_10_neighborhoods.hvplot.bar(x="neighborhood", y="sale_price_sqr_foot")

:Bars   [neighborhood]   (sale_price_sqr_foot)

Parallel Coordinates and Parallel Categories Analysis 

In [147]:
#Plot data using parallel_coordinates plot 

top_10_neighborhoods = top_10_neighborhoods.reset_index()
top_10_neighborhoods

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,903.993258,377427.50,2555.166667
1,Merced Heights,788.844818,380348.00,3414.000000
2,Miraloma Park,779.810842,375967.25,2155.250000
3,Pacific Heights,689.555817,378401.00,2817.285714
4,Westwood Park,687.087575,382295.00,3959.000000
5,Telegraph Hill,676.506578,378401.00,2817.285714
6,Presidio Heights,675.350212,378401.00,2817.285714
7,Cow Hollow,665.964042,378401.00,2817.285714
8,Potrero Hill,662.013613,378401.00,2817.285714
9,South Beach,650.124479,375805.00,2099.000000


In [148]:
px.parallel_coordinates(top_10_neighborhoods, color="gross_rent") 

In [152]:
#Parallel Categories Plot 

px.parallel_categories(
    top_10_neighborhoods,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="gross_rent",
    color_continuous_scale=px.colors.sequential.Inferno) 


Neighborhood Map 

In [122]:
#Load neighbohoods and coordinates data 

df_neighborhood_locations = pd.read_csv("neighborhoods_coordinates.csv")
df_neighborhood_locations.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


In [125]:
#Calculate the mean values for each neighborhood 

df3 = df3.reset_index()
df3

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,366.020712,378401.00,2817.285714
1,Anza Vista,373.382198,379050.00,3031.833333
2,Bayview,204.588623,376454.00,2318.400000
3,Bayview Heights,590.792839,382295.00,3739.000000
4,Bernal Heights,576.746488,379374.50,3080.333333
...,...,...,...,...
68,West Portal,498.488485,376940.75,2515.500000
69,Western Addition,307.562201,377427.50,2555.166667
70,Westwood Highlands,533.703935,376454.00,2250.500000
71,Westwood Park,687.087575,382295.00,3959.000000


In [128]:
#Join the average values with the neighborhood locations 

df4 = pd.concat([df3,df_neighborhood_locations], axis="columns", join="inner")
df4

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent,Neighborhood,Lat,Lon
0,Alamo Square,366.020712,378401.00,2817.285714,Alamo Square,37.791012,-122.402100
1,Anza Vista,373.382198,379050.00,3031.833333,Anza Vista,37.779598,-122.443451
2,Bayview,204.588623,376454.00,2318.400000,Bayview,37.734670,-122.401060
3,Bayview Heights,590.792839,382295.00,3739.000000,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,576.746488,379374.50,3080.333333,Bernal Heights,37.728630,-122.443050
...,...,...,...,...,...,...,...
68,West Portal,498.488485,376940.75,2515.500000,West Portal,37.740260,-122.463880
69,Western Addition,307.562201,377427.50,2555.166667,Western Addition,37.792980,-122.435790
70,Westwood Highlands,533.703935,376454.00,2250.500000,Westwood Highlands,37.734700,-122.456854
71,Westwood Park,687.087575,382295.00,3959.000000,Westwood Park,37.734150,-122.457000


Mapbox Visualization 

In [130]:
#Create a scatter mapbox to analyze neighborhood info 

map_1 = px.scatter_mapbox(
    df4,
    lat="Lat",
    lon="Lon",
    color="Neighborhood")

map_1.show()

In [131]:
#Create a scatter mapbox to analyze neighborhood info 

map_2 = px.scatter_mapbox(
    df4,
    lat="Lat",
    lon="Lon",
    color="sale_price_sqr_foot")

map_2.show()